## This notebook focus on merging manta, melt and survindel2 dataset from the 30x validation datasets


In [1]:
%%configure -f
{"driverMemory": "6000M"}

In [2]:
import hail as hl
hl.init(sc)

Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.1.2-amzn-0
SparkUI available at
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.80-4ccfae1ff293
LOGGING: writing to

In [3]:
## list all resources  used in this notebook 

release14_30x_melt_mt_uri =   "SG10K_SV_MELT_validation30x.n1922.m14025.30xvalidation.mt"
release14_30x_manta_mt_uri =   "SG10K_SV_MantaSVimmerGraphtyper_30x.n1922.m25313.discovery.DEL-INS-only.mt"
release14_30x_survindel_mt_uri = "SG10K_SV_SURVINDEL2.n1922.m21377.30xvalidation.mt"


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
manta_mt = hl.read_matrix_table(release14_30x_manta_mt_uri)
melt_mt = hl.read_matrix_table(release14_30x_melt_mt_uri)
surv_mt = hl.read_matrix_table(release14_30x_survindel_mt_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def align_mt2_cols_to_mt1(mt1, mt2):
    mt1 = mt1.add_col_index()
    mt2 = mt2.add_col_index()
    new_col_order = mt2.index_cols(mt1.col_key).col_idx.collect()
    new_mt = mt2.choose_cols(new_col_order)
    new_mt = new_mt.drop('col_idx')
    return new_mt

manta_mt = manta_mt
melt_mt = align_mt2_cols_to_mt1(manta_mt, melt_mt).checkpoint("SG10K_SV_MELT.n1922.m14025.30xvalidation.mt", overwrite=True)
surv_mt = align_mt2_cols_to_mt1(manta_mt, surv_mt).checkpoint("SG10K_SV_SURVINDEL2.n1922.m21377.30xvalidation.mt", overwrite=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2024-05-06 09:42:15 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2024-05-06 09:42:28 Hail: INFO: wrote matrix table with 14025 rows and 1922 columns in 13 partitions to SG10K_SV_MELT.n1922.m14025.30xvalidation.mt
2024-05-06 09:42:42 Hail: INFO: wrote matrix table with 21377 rows and 1922 columns in 9 partitions to SG10K_SV_SURVINDEL2.n1922.m21377.30xvalidation.mt

In [6]:
mt = manta_mt
mt = mt.annotate_entries(manta = hl.struct(), melt = hl.struct(), survindel2 = hl.struct())
mt = mt.transmute_entries(manta = mt.manta.annotate(**mt.entry), melt = hl.struct(), survindel2 = hl.struct())
mt = mt.annotate_entries(manta = mt.manta.drop('manta', 'melt', 'survindel2'))

mt = mt.annotate_rows(info = mt.info.annotate(alleles = mt.alleles, rsid = mt.rsid))
mt = mt.rename({'info':'info_manta'})
mt = mt.annotate_rows(info = hl.struct())
mt = mt.transmute_rows(info = mt.info.annotate(manta = mt.info_manta, melt = hl.struct(), survindel2 = hl.struct()))

manta_mt = mt
manta_mt.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        manta: struct {
            ABHet: float64, 
            ABHom: float64, 
            ABHetMulti: array<float64>, 
            ABHomMulti: array<float64>, 
            AC: array<int32>, 
            AF: array<float64>, 
            AN: int32, 
            CR: int32, 
            END: int32, 
            HOMSEQ: array<str>, 
            INV3: bool, 
            INV5: bool, 
            LEFT_SVINSSEQ: array<str>, 
            LOGF: float64, 
            MaxAAS: array<int32>, 
            MaxAASR: array<float64>, 
            MaxAltPP: int32, 
            MQ: int32, 
            MQsquared: int32, 
            NCLUSTERS: int32, 
            NGT: array<int32>, 
      

In [7]:
mt = melt_mt
mt = mt.annotate_entries(manta = hl.struct(), melt = hl.struct(), survindel2 = hl.struct())
mt = mt.transmute_entries(manta = hl.struct(), melt = mt.melt.annotate(**mt.entry), survindel2 = hl.struct())
mt = mt.annotate_entries(melt = mt.melt.drop('manta', 'melt', 'survindel2'))

mt = mt.annotate_rows(info = mt.info.annotate(alleles = mt.alleles, rsid = mt.rsid))
mt = mt.rename({'info':'info_melt'})
mt = mt.annotate_rows(info = hl.struct())
mt = mt.transmute_rows(info = mt.info.annotate(manta = hl.struct(), melt = mt.info_melt, survindel2 = hl.struct()))

melt_mt = mt
melt_mt.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        manta: struct {}, 
        melt: struct {
            ASSESS: int32, 
            TSD: str, 
            INTERNAL: array<str>, 
            SVTYPE: str, 
            SVLEN: int32, 
            MEINFO: array<str>, 
            DIFF: array<str>, 
            LP: int32, 
            RP: int32, 
            RA: float64, 
            PRIOR: str, 
            SR: int32, 
            alleles: array<str>, 
            rsid: str
        }, 
        survindel2: struct {}
    }
----------------------------------------
Entry fields:
    'manta': struct {}
    'melt': struct {
        GT: call, 
        GL: array<float64>, 
        DP: int32, 
        AD: int32, 
        PL: 

In [8]:
mt = surv_mt
mt = mt.annotate_entries(manta = hl.struct(), melt = hl.struct(), survindel2 = hl.struct())
mt = mt.transmute_entries(manta = hl.struct(), melt = hl.struct(), survindel2 = mt.survindel2.annotate(**mt.entry))
mt = mt.annotate_entries(survindel2 = mt.survindel2.drop('manta', 'melt', 'survindel2'))

mt = mt.annotate_rows(info = mt.info.annotate(alleles = mt.alleles, rsid = mt.rsid))
mt = mt.rename({'info':'info_survindel2'})
mt = mt.annotate_rows(info = hl.struct())
mt = mt.transmute_rows(info = mt.info.annotate(manta = hl.struct(), melt = hl.struct(),survindel2 = mt.info_survindel2))
mt = mt.drop('variant_qc','site_is_pass','n_called','n_hom_ref','n_het','n_hom_alt','n_non_ref')

surv_mt = mt
surv_mt.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        manta: struct {}, 
        melt: struct {}, 
        survindel2: struct {
            SVTYPE: str, 
            END: int32, 
            N_SAMPLES: int32, 
            N_SVS: int32, 
            GCF: float64, 
            SVSIZE: int32, 
            alleles: array<str>, 
            rsid: str
        }
    }
----------------------------------------
Entry fields:
    'manta': struct {}
    'melt': struct {}
    'survindel2': struct {
        GT: call, 
        FT: str, 
        AR: int32, 
        SR: int32, 
        RR: int32, 
        RS: int32, 
        RE: int32, 
        CN: int32, 
        IL: int32, 
        WR: int32, 
        DFL: int32, 
        DDL: int

In [9]:
#mt = manta_mt 
surv_row_ht = surv_mt.rows()
melt_row_ht = melt_mt.rows()
manta_row_ht = manta_mt.rows()


manta_mt = manta_mt.transmute_rows(
    info = manta_mt.info.annotate(
            # manta = manta_row_ht[manta_mt.row_key].info.manta,
            melt = melt_row_ht[manta_mt.row_key].info.melt,
            survindel2 = surv_row_ht[manta_mt.row_key].info.survindel2,
    ))
#manta_mt.describe()
manta_mt = manta_mt.annotate_entries(    
    #manta = manta_mt[manta_mt.row_key, manta_mt.col_key].manta,
    melt = melt_mt[manta_mt.row_key, manta_mt.col_key].melt,
    survindel2 = surv_mt[manta_mt.row_key, manta_mt.col_key].survindel2, 
)
#manta_mt.describe()
manta_mt = manta_mt.checkpoint("SG10K_SV_MantaSVimmerGraphtyper.n1922.m25313.30xvalidation.DEL-INS-only.complete_schema.mt", overwrite=True)
manta_mt.describe()



melt_mt = melt_mt.transmute_rows(
    info = melt_mt.info.annotate(
            manta = manta_row_ht[melt_mt.row_key].info.manta,
            # melt = melt_row_ht[melt_mt.row_key].info.melt,
            survindel2 = surv_row_ht[melt_mt.row_key].info.survindel2,
    ))
#melt_mt.describe()
melt_mt = melt_mt.annotate_entries(    
    manta = manta_mt[melt_mt.row_key, melt_mt.col_key].manta,
   # melt = melt_mt[melt_mt.row_key, melt_mt.col_key].melt,
    survindel2 = surv_mt[melt_mt.row_key, melt_mt.col_key].survindel2,
)
#melt_mt.describe()
melt_mt = melt_mt.checkpoint("SG10K_SV_MELT.n1922.m14025.30xvalidation.complete_schema.mt", overwrite=True)
melt_mt.describe()



surv_mt = surv_mt.transmute_rows(
    info = surv_mt.info.annotate(
            manta = manta_row_ht[surv_mt.row_key].info.manta,
            melt = melt_row_ht[surv_mt.row_key].info.melt,
            # survindel2 = surv_row_ht[surv_mt.row_key].info.survindel2,
    ))
#surv_mt.describe()
surv_mt = surv_mt.annotate_entries(    
    manta = manta_mt[surv_mt.row_key, surv_mt.col_key].manta,
    melt = melt_mt[surv_mt.row_key, surv_mt.col_key].melt,
   # survindel2 = surv_mt[surv_mt.row_key, surv_mt.col_key].survindel2,
)
#surv_mt.describe()
surv_mt = surv_mt.checkpoint("SG10K_SV_SURVINDEL2.n1922.m21377.30xvalidation.complete_schema.mt", overwrite=True)
surv_mt.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        manta: struct {
            ABHet: float64, 
            ABHom: float64, 
            ABHetMulti: array<float64>, 
            ABHomMulti: array<float64>, 
            AC: array<int32>, 
            AF: array<float64>, 
            AN: int32, 
            CR: int32, 
            END: int32, 
            HOMSEQ: array<str>, 
            INV3: bool, 
            INV5: bool, 
            LEFT_SVINSSEQ: array<str>, 
            LOGF: float64, 
            MaxAAS: array<int32>, 
            MaxAASR: array<float64>, 
            MaxAltPP: int32, 
            MQ: int32, 
            MQsquared: int32, 
            NCLUSTERS: int32, 
            NGT: array<int32>, 
      

In [10]:
mts = [manta_mt, melt_mt, surv_mt]
mt = mts.pop()
mt = mt.union_rows(*mts)
mt = mt.checkpoint("pre-merged.30xvalidation.mt", overwrite=True)


mt.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        manta: struct {
            ABHet: float64, 
            ABHom: float64, 
            ABHetMulti: array<float64>, 
            ABHomMulti: array<float64>, 
            AC: array<int32>, 
            AF: array<float64>, 
            AN: int32, 
            CR: int32, 
            END: int32, 
            HOMSEQ: array<str>, 
            INV3: bool, 
            INV5: bool, 
            LEFT_SVINSSEQ: array<str>, 
            LOGF: float64, 
            MaxAAS: array<int32>, 
            MaxAASR: array<float64>, 
            MaxAltPP: int32, 
            MQ: int32, 
            MQsquared: int32, 
            NCLUSTERS: int32, 
            NGT: array<int32>, 
      

In [11]:
mt = mt.annotate_rows(
    info = mt.info.annotate(
        SVTYPE = hl.coalesce(
            mt.info.manta.SVTYPE,
            hl.or_missing(hl.is_defined(mt.info.melt.SVTYPE), "INS"), #mt.info.melt.SVTYPE,
            mt.info.survindel2.SVTYPE
        ),
        SVSIZE = hl.coalesce(
            hl.if_else(mt.info.manta.SVSIZE > 0, mt.info.manta.SVSIZE, hl.missing(hl.tint32)),
            hl.if_else(mt.info.melt.SVLEN > 0, mt.info.melt.SVLEN, hl.missing(hl.tint32)),
            hl.if_else(mt.info.survindel2.SVSIZE > 0, mt.info.survindel2.SVSIZE, hl.missing(hl.tint32)),
        ),
        END = hl.coalesce(
            hl.if_else(mt.info.manta.END > 0, mt.info.manta.END, hl.missing(hl.tint32)),
            hl.if_else(mt.info.survindel2.END > 0, mt.info.survindel2.END, hl.missing(hl.tint32)),
        ),
    )
)

mt = mt.annotate_rows(
    rsid = "SG10K_SV_" 
               + mt.info.SVTYPE  + "_" 
               + mt.locus.contig + "_"
               + hl.str(mt.locus.position) + "_" 
               +  hl.or_else(hl.str(mt.info.SVSIZE), "UNDEF")
)
    
    
    
mt = mt.annotate_entries(
    GT = hl.coalesce(mt.manta.GT,mt.melt.GT,mt.survindel2.GT),
    DP = hl.coalesce(mt.manta.DP,mt.melt.DP),
#    AD = hl.coalesce(mt.manta.AD,mt.melt.AD),
    PL = hl.coalesce(mt.manta.PL,mt.melt.PL),
)
 
mt.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        manta: struct {
            ABHet: float64, 
            ABHom: float64, 
            ABHetMulti: array<float64>, 
            ABHomMulti: array<float64>, 
            AC: array<int32>, 
            AF: array<float64>, 
            AN: int32, 
            CR: int32, 
            END: int32, 
            HOMSEQ: array<str>, 
            INV3: bool, 
            INV5: bool, 
            LEFT_SVINSSEQ: array<str>, 
            LOGF: float64, 
            MaxAAS: array<int32>, 
            MaxAASR: array<float64>, 
            MaxAltPP: int32, 
            MQ: int32, 
            MQsquared: int32, 
            NCLUSTERS: int32, 
            NGT: array<int32>, 
      

In [12]:
## make sure no duplicate rsid
def rename_duplicate_rows(dataset, field='rsid', name=None, override=False):
    """Rename duplicated row-wide field
       inspired from https://hail.is/docs/0.2/methods/misc.html#hail.methods.rename_duplicates
    """

    if (override is True) and (name is not None):
        hl.utils.java.info('rename_duplicate_rows cannnot specfiy both a new field name and overrride the exisiting one')
        raise ValueError('rename_duplicate_rows cannnot specfiy both a new field name and overrride the exisiting one')

    if (override is False) and (name is None):
        name = 'uniq_' + field

    ids = dataset.aggregate_rows(hl.agg.collect(dataset[field]))
    mapping, new_ids = hl.utils.deduplicate(ids)
    if mapping:
        hl.utils.java.info(
            f'Renamed {len(mapping)} duplicate `{field}` {hl.utils.misc.plural("field", len(mapping))} as `{name}`. Mangled IDs as follows:'
            + ''.join(f'\n  "{pre}" => "{post}"' for pre, post in mapping)
        )
    else:
        hl.utils.java.info('No duplicate field found.')

    return dataset.annotate_rows(**{name: hl.literal(new_ids)[hl.int(hl.scan.count())]})

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
ori_mt = mt
mt = mt.rename({'rsid':'_old_sv_id',
                'info' : 'callers_info'
                })
mt = rename_duplicate_rows(mt, field='_old_sv_id', name='sv_id')
mt.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    '_old_sv_id': str
    'qual': float64
    'filters': set<str>
    'callers_info': struct {
        manta: struct {
            ABHet: float64, 
            ABHom: float64, 
            ABHetMulti: array<float64>, 
            ABHomMulti: array<float64>, 
            AC: array<int32>, 
            AF: array<float64>, 
            AN: int32, 
            CR: int32, 
            END: int32, 
            HOMSEQ: array<str>, 
            INV3: bool, 
            INV5: bool, 
            LEFT_SVINSSEQ: array<str>, 
            LOGF: float64, 
            MaxAAS: array<int32>, 
            MaxAASR: array<float64>, 
            MaxAltPP: int32, 
            MQ: int32, 
            MQsquared: int32, 
            NCLUSTERS: int32, 
            NGT: array<i

In [14]:
mt.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(60715, 1922)

In [15]:
## write the resulting mt
release14_30xvalidation_mt_uri =   "SG10K-SV-Release-1.4-30x-validation.mt"
mt.write(release14_30xvalidation_mt_uri, overwrite=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2024-05-06 09:54:56 Hail: INFO: wrote matrix table with 60715 rows and 1922 columns in 46 partitions to sSG10K-SV-Release-1.4-30x-validation.mt
    Total size: 1.26 GiB
    * Rows/entries: 1.26 GiB
    * Columns: 6.94 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  2576 rows (59.87 MiB)